In [12]:
import pandas as pd

In [13]:
fractional_cover = pd.read_csv("datasets/fractional_cover.csv")
spectra_forest_floor = pd.read_csv("datasets/spectra_forest_floor.csv")
complete_fc = pd.DataFrame(columns=["plot_ID", "location", "vasc", "nonvasc", "lichen", "intactlitt", "decomplitt"])

In [14]:
def infer_value_linearly(x: int, x1: int, x2: int, v1: float, v2: float) -> float:
    a = (v2 - v1) / (x2 - x1)
    b = (x2 * v1 - x1 * v2) / (x2 - x1)

    return a * x + b

Generate missing fractional cover samples linearly from the values of the two closest locations from the set {1, 5, 9, 13}. For locations 14 and 15, the values from location 13 are used directly.

In [15]:
for i, sample in fractional_cover.iterrows():
    for loc in range(1, 16):
        if sample['plot_ID'] != 'HY_PINE5' or loc != 15:
            if loc > 13:
                complete_fc.loc[len(complete_fc)] = {
                    "plot_ID": sample["plot_ID"],
                    "location": loc,
                    "vasc": sample["vasc_q4"],
                    "nonvasc": sample["nonvasc_q4"],
                    "lichen": sample["lichen_q4"],
                    "intactlitt": sample["intactlitt_q4"],
                    "decomplitt": sample["decomplitt_q4"]
                }
            else:
                if loc <= 5:
                    point1 = 1
                    point2 = 2
                    loc1 = 1
                    loc2 = 5

                elif loc <= 9:
                    point1 = 2
                    point2 = 3
                    loc1 = 5
                    loc2 = 9

                elif loc <= 13:
                    point1 = 3
                    point2 = 4
                    loc1 = 9
                    loc2 = 13

                complete_fc.loc[len(complete_fc)] = {
                    "plot_ID": sample["plot_ID"],
                    "location": loc,
                    "vasc": infer_value_linearly(loc, loc1, loc2, sample[f"vasc_q{point1}"], sample[f"vasc_q{point2}"]),
                    "nonvasc": infer_value_linearly(loc, loc1, loc2, sample[f"nonvasc_q{point1}"], sample[f"nonvasc_q{point2}"]),
                    "lichen": infer_value_linearly(loc, loc1, loc2, sample[f"lichen_q{point1}"], sample[f"lichen_q{point2}"]),
                    "intactlitt": infer_value_linearly(loc, loc1, loc2, sample[f"intactlitt_q{point1}"], sample[f"intactlitt_q{point2}"]),
                    "decomplitt": infer_value_linearly(loc, loc1, loc2, sample[f"decomplitt_q{point1}"], sample[f"decomplitt_q{point2}"]),
                }

In [16]:
complete_fc

,plot_ID,location,vasc,nonvasc,lichen,intactlitt,decomplitt
0,HY_BIRCH1,1,0.410,0.04,0.0,0.550,0.0
1,HY_BIRCH1,2,0.475,0.03,0.0,0.495,0.0
2,HY_BIRCH1,3,0.540,0.02,0.0,0.440,0.0
3,HY_BIRCH1,4,0.605,0.01,0.0,0.385,0.0
4,HY_BIRCH1,5,0.670,0.00,0.0,0.330,0.0
...,...,...,...,...,...,...,...
864,LZ_OAK4,11,0.420,0.00,0.0,0.580,0.0
865,LZ_OAK4,12,0.530,0.00,0.0,0.470,0.0
866,LZ_OAK4,13,0.640,0.00,0.0,0.360,0.0
867,LZ_OAK4,14,0.640,0.00,0.0,0.360,0.0


In [17]:
complete_fc = complete_fc.drop(columns=["plot_ID", "location"])
complete_fc.to_csv("datasets/complete_fc.csv", sep=",", index=False)